## 数据预处理技术

**torchvision-计算机视觉工具包**

- torchvision.transforms

    常用的图像预处理方法
    
- torchvision.dataset

    MNIST,CIFAR-10,ImageNet图像数据集
    
- torchvision.model

    常用的预训练模型：AlexNet，VGG，ResNet，GoogLeNet


**常用的图像预处理方法**

- 数据中心化

- 数据标准化

- 缩放

- 裁剪

- 旋转

- 翻转

- 填充

- 噪声添加

- 灰度变换

- 线性变换

- 仿射变换

- 亮度、饱和度和对比度变换

**数据标准化**

    transforms.Normalize(mean,std,inplace=False)

- 功能

    逐通道对图像像素归一化
    $$out=(input-mean)/std$$

- mean

    各通道的均值
    
- std

    各通道的方差
    
- inplace

    是否原地操作
    

    代码使用：
        norm_mean = [0.485, 0.456, 0.406]
        norm_std = [0.229, 0.224, 0.225]
        
        transform = transforms.Compose([
        transforms.Normalize(norm_mean, norm_std)
        ])

## 数据增强

 **图像剪裁**
 
     transforms.CenterCrop(size)
 
- 功能
 
     从图像中心裁剪图像
     
- size

    所需裁剪图片尺寸
    
- 注意：

    1.当图片尺寸大于size时，则会对原图片进行裁剪
    
    2.当图片尺寸小于size时，图片会填充（可用于细粒度不同大小的图像分类）

    代码使用：
        
        transform = transforms.Compose([
        transforms.CenterCrop(512)
        ])

    transforms.Resize((a,b))

- 功能

    将图片尺寸变为(a,b)
    

    transforms.RandomCrop(size,padding=None,pad_if_needed=False,fill=0,padding_mode='constant')

- 功能

    从图片中随机剪裁处尺寸为size的图片

- padding

    设置填充大小 
    - a:上下左右填充a个像素
    - (a,b) 上下填充b个像素，左右填充a个像素
    - (a,b,c,d) 左：a  上：b  右：c  下：d

- pad_if_needed

    若图像小于设定size，则填充
    
- padding_mode

    - 填充模式
    
    - 1. constant  像素值由fill指定
    
    - 2. edge  像素值由图像边缘像素得到
    
    - 3. reflect  镜像填充，最后一个像素不镜像
    
    - 4. symmetric  镜像填充，最后一个像素填充
    
- fill

    constant时设置的像素值

    代码实现：
        
        # 2 RandomCrop
        transforms.RandomCrop(224, padding=16)  
        transforms.RandomCrop(224, padding=(16, 64))
        transforms.RandomCrop(224, padding=16, fill=(255, 0, 0))
        transforms.RandomCrop(512, pad_if_needed=True)   
        # pad_if_needed=True，尺寸大于图像否则报错
        transforms.RandomCrop(224, padding=64, padding_mode='edge')
        transforms.RandomCrop(224, padding=64, padding_mode='reflect')
        transforms.RandomCrop(1024, padding=1024, padding_mode='symmetric')

    transforms.RandomResizedCrop(size,scale=(0.08,1.0),ratio=(3/4,4/3),interpolation)

- 功能

    随机大小，长宽比，裁剪图片
  
- size

    所需剪裁的尺寸大小

- scale

    随机剪裁面积比例，默认区间（0.08,1）
    
- ratio

    随机长宽比，默认区间（3/4,4/3）
    
- interpolation (插值方法)

    - PIL.Image.NEAREST
    
    - PIL.Image.BILINEAR
    
    - PIL.Image.BICUBIC
    

    代码实现：
    
        transforms.RandomResizedCrop(size=224, scale=(0.5, 0.5))

    transforms.FiveCrop(size)
    
    transforms.TenCrop(size,vertical_flip=False)
    
- 功能

    FiveCrop对图像的上下左右以及中心裁剪出5张图片，TenCrop对这5张图片进行水平和垂直镜像，得到10张图片。
    
- size

    所需剪裁的尺寸
    
- vertical_flip

    选择是否垂直翻转
    

    代码实现：
    
        transforms.FiveCrop(112),
        transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops]))
        ##注意由于FiveCrop输出的不是张量，因此要用Lambda函数进行张量转化

    transforms.TenCrop(112, vertical_flip=False),
    transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops]))

**翻转，反转**

    transforms.RandomHorizontalFlip(p=0.5)

    transforms.RandomVerticalFlip(p=0.5)
- 功能

    依概率p对图片进行水平或者垂直翻转
    
- p

    翻转概率

    代码实现：
    
           # 1 Horizontal Flip
           transforms.RandomHorizontalFlip(p=1),

           # 2 Vertical Flip
           transforms.RandomVerticalFlip(p=0.5)

**旋转**

    transforms.RandomRotation(degrees,resample=False,expand=False,center=None)

- 功能

    随机旋转图片
    
    a：表示在（-a，a）之间选择旋转角度
    
    （a,b）:在（a,b）之间选择角度
    
- resample

    重采样方法

- expand

    是否扩大图片，保持原图信息

- center

    旋转坐标

    代码实现：
    
         # 3 RandomRotation
         # transforms.RandomRotation(90),
         # transforms.RandomRotation((90), expand=True),
         # transforms.RandomRotation(30, center=(0, 0)),
         # transforms.RandomRotation(30, center=(0, 0), expand=True),   
         # expand only for center rotation

## 分类实战案例：

In [ ]:
import os
import numpy as np
import torch
import random
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tools.my_dataset import RMBDataset
from PIL import Image
from matplotlib import pyplot as plt


def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


set_seed(1)  # 设置随机种子

# 参数设置
MAX_EPOCH = 10
BATCH_SIZE = 1
LR = 0.01
log_interval = 10
val_interval = 1
rmb_label = {"1": 0, "100": 1}


def transform_invert(img_, transform_train):
    """
    将data 进行反transfrom操作
    :param img_: tensor
    :param transform_train: torchvision.transforms
    :return: PIL image
    """
    if 'Normalize' in str(transform_train):
        norm_transform = list(filter(lambda x: isinstance(x, transforms.Normalize), transform_train.transforms))
        mean = torch.tensor(norm_transform[0].mean, dtype=img_.dtype, device=img_.device)
        std = torch.tensor(norm_transform[0].std, dtype=img_.dtype, device=img_.device)
        img_.mul_(std[:, None, None]).add_(mean[:, None, None])

    img_ = img_.transpose(0, 2).transpose(0, 1)  # C*H*W --> H*W*C
    img_ = np.array(img_) * 255

    if img_.shape[2] == 3:
        img_ = Image.fromarray(img_.astype('uint8')).convert('RGB')
    elif img_.shape[2] == 1:
        img_ = Image.fromarray(img_.astype('uint8').squeeze())
    else:
        raise Exception("Invalid img shape, expected 1 or 3 in axis 2, but got {}!".format(img_.shape[2]) )

    return img_


# ============================ step 1/5 数据 ============================
split_dir = os.path.join("..", "..", "data", "rmb_split")
train_dir = os.path.join(split_dir, "train")
valid_dir = os.path.join(split_dir, "valid")

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),

    # 1 CenterCrop
    # transforms.CenterCrop(512),     # 512

    # 2 RandomCrop
    # transforms.RandomCrop(224, padding=16),
    # transforms.RandomCrop(224, padding=(16, 64)),
    # transforms.RandomCrop(224, padding=16, fill=(255, 0, 0)),
    # transforms.RandomCrop(512, pad_if_needed=True),   # pad_if_needed=True
    # transforms.RandomCrop(224, padding=64, padding_mode='edge'),
    # transforms.RandomCrop(224, padding=64, padding_mode='reflect'),
    # transforms.RandomCrop(1024, padding=1024, padding_mode='symmetric'),

    # 3 RandomResizedCrop
    # transforms.RandomResizedCrop(size=224, scale=(0.5, 0.5)),

    # 4 FiveCrop
    # transforms.FiveCrop(112),
    # transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops])),

    # 5 TenCrop
    # transforms.TenCrop(112, vertical_flip=False),
    # transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops])),

    # 1 Horizontal Flip
    # transforms.RandomHorizontalFlip(p=1),

    # 2 Vertical Flip
    # transforms.RandomVerticalFlip(p=0.5),

    # 3 RandomRotation
    # transforms.RandomRotation(90),
    # transforms.RandomRotation((90), expand=True),
    # transforms.RandomRotation(30, center=(0, 0)),
    # transforms.RandomRotation(30, center=(0, 0), expand=True),   # expand only for center rotation

    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# 构建MyDataset实例
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)

# 构建DataLoder
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)


# ============================ step 5/5 训练 ============================
for epoch in range(MAX_EPOCH):
    for i, data in enumerate(train_loader):

        inputs, labels = data   # B C H W

        img_tensor = inputs[0, ...]     # C H W
        img = transform_invert(img_tensor, train_transform)
        plt.imshow(img)
        plt.show()
        plt.pause(0.5)
        plt.close()

        # bs, ncrops, c, h, w = inputs.shape
        # for n in range(ncrops):
        #     img_tensor = inputs[0, n, ...]  # C H W
        #     img = transform_invert(img_tensor, train_transform)
        #     plt.imshow(img)
        #     plt.show()
        #     plt.pause(1)